In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, IntegerType, DateType, TimestampType, FloatType 
catalog_name = 'ecommerce'

In [0]:
df = spark.table(f"{catalog_name}.silver.slv_order_items")

df.limit(10).display()

In [0]:
df.printSchema()

In [0]:
# 1) Add gross amount
df = df.withColumn(
    "gross_amount",
    F.col("quantity") * F.col("unit_price")
    )

# 2) Add discount_amount (discount_pct is already numeric, e.g., 21 -> 21%)
df = df.withColumn(
    "discount_amount",
    F.ceil(F.col("gross_amount") * (F.col("discount_pct") / 100.0))
)

# 3) Add sale_amount = gross - discount
df = df.withColumn(
    "sale_amount",
    F.col("gross_amount") - F.col("discount_amount") + F.col("tax_amount")
)

# adding date id makes joins, partitioning, and analytics faster, cleaner, and fully consistent.
df = df.withColumn("date_id", F.date_format(F.col("date"), "yyyyMMdd").cast(IntegerType()))  

#  coupon flag = 1 if coupon_code is not null else 0
df = df.withColumn(
    'coupon_flag',
    F.when(
        F.col('coupon_code').isNotNull()
        ,F.lit(1)
        ).otherwise(F.lit(0))
    )


df.limit(10).display()    

In [0]:
# Count null values in coupon code 
df.select(F.sum(F.when(F.col('coupon_code').isNull(),1).otherwise(0)).alias('coupon_code_null')).show()

In [0]:
# --- 1) Fixed FX rates---
fx_rates = {
    "INR": 1.00,
    "AED": 24.18,
    "AUD": 57.55,
    "CAD": 62.93,
    "GBP": 117.98,
    "SGD": 68.18,
    "USD": 88.29,
}

rates = [(k, float(v)) for k, v in fx_rates.items()]
rates_df = spark.createDataFrame(rates, ["currency", "fx_inr_rate"])
rates_df.show()

In [0]:
# Join FX rates to convert sales from various currencies to INR and round up the converted amount

df = (
    df
    .join(
        rates_df,
        rates_df.currency == F.upper(F.trim(F.col("unit_price_currency"))),
        "left"
    )
    .withColumn("sale_amount_inr", F.col("sale_amount") * F.col("fx_inr_rate"))
    .withColumn("sale_amount_inr", F.ceil(F.col("sale_amount_inr")))
)

In [0]:
# Column descriptions for selected DataFrame:
# date_id: Integer date identifier in format yyyyMMdd
# transaction_date: Date of the transaction
# transaction_ts: Timestamp of the transaction
# transaction_id: Unique order identifier
# customer_id: Unique customer identifier
# seq_no: Sequence number of the item in the order
# product_id: Unique product identifier
# channel: Sales channel (e.g., online, offline)
# coupon_code: Applied coupon code, if any
# coupon_flag: 1 if coupon_code is present, else 0
# unit_price_currency: Currency code for unit price
# quantity: Number of items ordered
# unit_price: Price per unit in unit_price_currency
# gross_amount: Total price before discount and tax
# discount_percent: Discount percentage applied
# discount_amount: Discount value applied
# tax_amount: Tax value applied
# net_amount: Final amount after discount and tax (in original currency)
# net_amount_inr: Final amount after discount and tax, converted to INR

df = df.select(
    F.col("date_id"),
    F.col("date").alias("transaction_date"),
    F.col("order_ts").alias("transaction_ts"),
    F.col("order_id").alias("transaction_id"),
    F.col("customer_id"),
    F.col("item_seq").alias("seq_no"),
    F.col("product_id"),
    F.col("channel"),
    F.col("coupon_code"),
    F.col("coupon_flag"),
    F.col("unit_price_currency"),
    F.col("quantity"),
    F.col("unit_price"),
    F.col("gross_amount"),
    F.col("discount_pct").alias("discount_percent"),
    F.col("discount_amount"),
    F.col("tax_amount"),
    F.col("sale_amount").alias("net_amount"),
    F.col("sale_amount_inr").alias("net_amount_inr")
)

In [0]:
display(df.limit(10))

In [0]:
# Write raw data to the gold layer (catalog: ecommerce, schema: gold, table: gld_fact_order_items)
df.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(f"{catalog_name}.gold.gld_fact_order_items")

In [0]:
spark.sql(f"SELECT count(*) FROM {catalog_name}.gold.gld_fact_order_items").show()